# Task 4: Analysing building density
**The task is to link the IÖR-Monitor indicator "Building density in reference area" with the synthetic SOEP Structural Dataset of Berlin, for two years: 2014 and 2021**

## Load functions from SoRa R package
This steps are currently required to load all R functions from /R/ directory. In future, the SoRa R package will be installed directly.

In [ ]:
# load R functions from SoRa R Package
path <- "/home/jovyan/R/"
sora_functions  <- dir(path)
for (i in sora_functions) {
  source(paste0(path, i))
}

## Check your changed SORA_API_KEY 
- the environment variable from .Renviron file


In [ ]:
#check environment variable for SORA_API_KEY
Sys.getenv("SORA_API_KEY")

## Load, explore and prepare input survey data
- SOEP structural dataset are synthetic coordinates with a similar spatial distribution in comparison to the origin coordinates of the survey
- SOEP structural dataset has ID and syear columns, which bot are needed to create a  unique ID

In [ ]:
## load data
path_data <- "/home/jovyan/data/"
## data Berlin
berlin <- read.csv(paste0(path_data, "berlin.csv"))

## subset for coordinates of 2014
berlin_2014 <- berlin[berlin$syear == 2014, ]
berlin_2014 <- sora_assemble_id(berlin_2014, id_col = c("id", "syear"))

## subset for coordinates of 2021
berlin_2021 <- berlin[berlin$syear == 2021, ]
berlin_2021 <- sora_assemble_id(berlin_2021, id_col = c("id", "syear"))

Explore the input datasets

In [ ]:
head(berlin_2014)

In [ ]:
## plot berlin in 2014
plot(berlin_2014$x, berlin_2014$y, 
     xlab = "x", ylab = "y",
     main = "Households in Berlin - in 2014",
     sub = "crs = 4647")
grid()

In [ ]:
head(berlin_2021)

In [ ]:
## plot berlin in 2021
plot(berlin_2021$x, berlin_2021$y, 
     xlab = "x", ylab = "y",
     main = "Households in Berlin - in 2021",
     sub = "crs = 4647")
grid()

In [ ]:
## check if sora is available
## script stops here, if there is a problem!

stopifnot(sora_available())

### Linking for 2014
prepare and execute linking job for year 2014

In [ ]:
sora_data_2014 <- sora_custom(.data = berlin_2014, crs = 4647)

In [ ]:
# define geospatial dataset for linkage
spat_data_2014 <- sora_spatial(id = "ioer-monitor-g01dg-2014-1000m")

In [ ]:
linking_2014 <- sora_linking(
  method = "<--->",
  selection_area = "<--->",
  length = <--->,
  output = c("mean", "min", "max", "sd")
)

Start the linking job

In [ ]:
job_id_2014 <- sora_request(dataset = sora_data_2014, link_to = spat_data_2014, method = linking_2014)

### Get results

Check if linking job is already finished

In [ ]:
## check, if your current linking job is done
sora_job_done(job_id_2014)

Or check the job status of all your requested linking jobs

In [ ]:
## check status of all your started linking jobs
sora_jobs()

**If your linking job is finished**, then you can get the result data

In [ ]:
## get results
sora_calls <- sora_jobs()
job_id_2014 <- sora_calls$job_id[1]

if (sora_job_done(job_id_2014)){
  results_2014 <- sora_results(job_id = job_id_2014)
  results_2014 <- sora_split_id(results_2014, out_col = c("hid", "syear"))
  head(results_2014)
}


### Linking for 2021
prepare and execute linking job for year 2021
- hint: 
    - to determine the correct linking parameter, you can either use the online web-interface of the datapicker
    - or read the error messages (after starting a linking job), to get an idea which values are valid

In [ ]:
sora_data_2021 <- sora_custom(.data = berlin_2021, crs = 4647)

In [ ]:
# define geospatial dataset for linkage
spat_data_2021 <- sora_spatial(id = "<--->")

In [ ]:
linking_2021 <- sora_linking(
  method = "<--->",
  selection_area = "<--->",
  length = <--->,
  output = c("mean", "min", "max", "sd")
)

In [ ]:
job_id_2021 <- sora_request(dataset = sora_data_2021, link_to = spat_data_2021, method = linking_2021)

### Get results

In [ ]:
## check, if your current linking job is done
sora_job_done(job_id_2021)

In [ ]:
## check status of all your started linking jobs
sora_jobs()

In [ ]:
## get results
sora_calls <- sora_jobs()
job_id_2021 <- sora_calls$job_id[1]

if (sora_job_done(job_id_2021)){
  results_2021 <- sora_results(job_id = job_id_2021)
  results_2021 <- sora_split_id(results_2021, out_col = c("hid", "syear"))
  head(results_2021)
}


### Analyses
- for both years (2014 and 2021) we have synthetic SOEP Structural Dataset coordinates, we linked now with geospatial dataset
- e.g. we could compare the linked values of both years
- **in the future**:
    - you could request a Data Use Contract with SOEP to get SOEPcore (without coordinates) + SOEP Structural Dataset (synthetic coordinates with similar spatial distribution) to execute your own test linking jobs from home (Public mode in SoRa)
    - you could apply for a research stay in Berlin using the secure room of SOEP to link the origin coordinates of SOEP with SOEPcore. Offering the best data for spatial linking (Private mode in SoRa)

In [ ]:
## find only id which exist in both subsets '2014' and '2021'
unique_id <- c(results_2014$hid, results_2021$hid)
unique_id <- unique_id[which(duplicated(unique_id) == TRUE)]

In [ ]:
## select data for year 2014
analyse_id <- which(is.element(results_2014$hid, unique_id) == TRUE)
analyse_results_2014 <- results_2014[analyse_id,]

In [ ]:
## select data for year 2021
analyse_id <- which(is.element(results_2021$hid, unique_id) == TRUE)
analyse_results_2021 <- results_2021[analyse_id,]

In [ ]:
## plot difference
diff_mean <- analyse_results_2014$mean - analyse_results_2021$mean
data_diff <- merge(results_2014, results_2021, by = "hid")
data_diff$diff_mean <- diff_mean
names(data_diff) <- gsub(".x", "_2014", names(data_diff), fixed = TRUE)
names(data_diff) <- gsub(".y", "_2014", names(data_diff), fixed = TRUE)

color_blue <- which(diff_mean < 0)
use_color <- rep("red", length(diff_mean))
use_color[color_blue] <- "blue"
range_y <- max(abs(diff_mean))
plot(diff_mean, col = use_color,
    main = "difference of mean for the years 2014 and 2021",
    xlab = "index hid, sorted by hid",
    ylab = "diff mean",
    ylim = c((range_y * -1), range_y))
abline(h = 0, lty = 2)
head(data_diff)